Connection

In [2]:
import pandas as pd
from sqlalchemy import create_engine

In [3]:
connection_string = "postgres:postgres@localhost:5432/indiana_covid"
engine = create_engine(f'postgresql://{connection_string}')

In [4]:
# Confirm tables
engine.table_names()

['cases_and_deaths', 'hospital']

Joined Tables

In [10]:
join_query= """
                    SELECT
                       cases_and_deaths.county,
                       cases_and_deaths.covid_cases,
                       cases_and_deaths.covid_deaths,
                       hospital.hospitalization_count
                    FROM
                        hospital
                        
                    INNER JOIN cases_and_deaths ON
                        cases_and_deaths.county= hospital.county
                    
                
                       """

In [11]:
pd.read_sql_query(join_query, con= engine)

,county,covid_cases,covid_deaths,hospitalization_count
0,Adams,1959,24,155
1,Allen,21054,342,1469
2,Bartholomew,3915,66,330
3,Benton,465,3,40
4,Blackford,673,17,65
...,...,...,...,...
86,Washington,916,9,73
87,Wayne,3702,96,360
88,Wells,1471,36,115
89,White,1562,24,138


CASES TO HOSPITAL

In [24]:
hc_query= """
                    SELECT CAST( 1 As Decimal)/2
                       hospital.hospitalization_count/
                       
                        cases_and_deaths.covid_cases
                    FROM
                        hospital
                        
                    INNER JOIN cases_and_deaths ON
                        cases_and_deaths.county= hospital.county
                    
                
                       """

In [25]:
pd.read_sql_query(hc_query, con= engine)

ProgrammingError: (psycopg2.errors.SyntaxError) syntax error at or near "."
LINE 3:                        hospital.hospitalization_count/
                                       ^

[SQL: 
                    SELECT CAST( 1 As Decimal)/2
                       hospital.hospitalization_count/
                       
                        cases_and_deaths.covid_cases
                    FROM
                        hospital
                        
                    INNER JOIN cases_and_deaths ON
                        cases_and_deaths.county= hospital.county
                    
                
                       ]
(Background on this error at: http://sqlalche.me/e/13/f405)